# Day 16: Asset Allocation Strategies

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class4_taxes_portfolio/week4_portfolio_risk/day16_asset_allocation.ipynb)

## Learning Objectives

By the end of this lesson, you will be able to:

1. Apply age-based allocation rules (100 minus age, 110/120 variants)
2. Assess personal risk tolerance and time horizon
3. Calculate and interpret asset class correlations
4. Design diversified portfolio allocations using Modern Portfolio Theory concepts
5. Visualize portfolio compositions and evaluate risk/return tradeoffs

## Introduction

Asset allocation is arguably the most important investment decision you'll make. Studies show that asset allocation explains over 90% of portfolio return variance - more than individual security selection or market timing.

In this lesson, we'll explore:
- Traditional allocation frameworks
- Risk tolerance assessment
- Correlation and diversification
- Practical portfolio construction

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Part 1: Age-Based Allocation Rules

### The Classic Rules

Traditional asset allocation often starts with simple age-based formulas:

```
Age-Based Allocation Rules:
============================

1. "100 Minus Age" Rule (Conservative)
   Stocks % = 100 - Your Age
   Bonds % = Your Age
   
   Example: Age 30
   Stocks: 70%
   Bonds: 30%

2. "110 Minus Age" Rule (Moderate)
   Stocks % = 110 - Your Age
   Bonds % = Your Age - 10
   
   Example: Age 30
   Stocks: 80%
   Bonds: 20%

3. "120 Minus Age" Rule (Aggressive)
   Stocks % = 120 - Your Age
   Bonds % = Your Age - 20
   
   Example: Age 30
   Stocks: 90%
   Bonds: 10%
```

### Why These Rules Evolved

The original "100 minus age" rule was developed when:
- Life expectancy was shorter
- Bond yields were higher (5-7%)
- Stock market risk premiums were higher

Today, with:
- Longer lifespans (30+ year retirements)
- Lower bond yields (2-4%)
- Better diversification tools

Many advisors recommend the 110 or 120 variants for most investors.

In [ ]:
def calculate_age_based_allocation(age, rule='110'):
    """
    Calculate asset allocation based on age-based rules.
    
    Parameters:
    -----------
    age : int
        Investor's age
    rule : str
        '100', '110', or '120' for different risk levels
    
    Returns:
    --------
    dict : Dictionary with stock and bond percentages
    """
    rule_value = int(rule)
    stock_pct = max(0, min(100, rule_value - age))
    bond_pct = 100 - stock_pct
    
    return {
        'stocks': stock_pct,
        'bonds': bond_pct,
        'rule': f"{rule} minus age"
    }

# Example: 30-year-old investor
age = 30
print(f"Asset Allocation for {age}-Year-Old Investor\n" + "="*50)

for rule in ['100', '110', '120']:
    allocation = calculate_age_based_allocation(age, rule)
    print(f"\n{allocation['rule']}:")
    print(f"  Stocks: {allocation['stocks']}%")
    print(f"  Bonds:  {allocation['bonds']}%")

In [ ]:
# Visualize how allocation changes with age
ages = np.arange(20, 81, 5)
rules = ['100', '110', '120']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Stock allocation by age
for rule in rules:
    stock_allocations = [calculate_age_based_allocation(age, rule)['stocks'] 
                        for age in ages]
    ax1.plot(ages, stock_allocations, marker='o', linewidth=2, label=f"{rule} Rule")

ax1.set_xlabel('Age', fontsize=12, fontweight='bold')
ax1.set_ylabel('Stock Allocation (%)', fontsize=12, fontweight='bold')
ax1.set_title('Stock Allocation vs Age\nAcross Different Rules', 
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 105])

# Plot 2: Glide path visualization for 110 rule
ages_detailed = np.arange(25, 76, 1)
allocations = [calculate_age_based_allocation(age, '110') for age in ages_detailed]
stocks = [a['stocks'] for a in allocations]
bonds = [a['bonds'] for a in allocations]

ax2.fill_between(ages_detailed, 0, stocks, alpha=0.6, label='Stocks', color='#2E7D32')
ax2.fill_between(ages_detailed, stocks, 100, alpha=0.6, label='Bonds', color='#1565C0')

# Highlight key ages
key_ages = [30, 45, 60, 70]
for key_age in key_ages:
    alloc = calculate_age_based_allocation(key_age, '110')
    ax2.axvline(key_age, color='red', linestyle='--', alpha=0.5)
    ax2.text(key_age, 50, f"{alloc['stocks']}% stocks\nat age {key_age}", 
            rotation=90, verticalalignment='center', fontsize=9)

ax2.set_xlabel('Age', fontsize=12, fontweight='bold')
ax2.set_ylabel('Portfolio Allocation (%)', fontsize=12, fontweight='bold')
ax2.set_title('Target Date Glide Path\n110 Minus Age Rule', 
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=11, loc='right')
ax2.set_ylim([0, 100])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Takeaway:")
print("The 'glide path' shows how your allocation automatically becomes")
print("more conservative as you age, reducing volatility near retirement.")

## Part 2: Risk Tolerance Assessment

Age-based rules are a starting point, but true asset allocation must consider:

```
Risk Tolerance Factors:
=======================

1. Time Horizon
   - Years until you need the money
   - Longer horizon = more stock allocation
   
2. Risk Capacity
   - Financial ability to withstand losses
   - Job security, emergency fund, other assets
   
3. Risk Willingness
   - Emotional tolerance for volatility
   - Sleep-at-night factor
   
4. Financial Goals
   - Retirement, home purchase, education
   - Required vs desired returns
```

### The Risk Tolerance Questionnaire

In [ ]:
def assess_risk_tolerance():
    """
    Interactive risk tolerance assessment.
    In a real application, this would collect user input.
    """
    # Sample scores (in practice, these would be user inputs)
    questions = {
        'time_horizon': {
            'question': 'Years until you need this money?',
            'options': {
                '<5 years': 1,
                '5-10 years': 3,
                '10-20 years': 5,
                '20+ years': 7
            },
            'weight': 0.3
        },
        'portfolio_drop': {
            'question': 'If your $100k portfolio dropped to $80k, you would:',
            'options': {
                'Sell everything': 1,
                'Sell some': 3,
                'Hold steady': 5,
                'Buy more': 7
            },
            'weight': 0.25
        },
        'income_stability': {
            'question': 'Your income stability:',
            'options': {
                'Uncertain': 1,
                'Variable': 3,
                'Stable': 5,
                'Very stable': 7
            },
            'weight': 0.20
        },
        'emergency_fund': {
            'question': 'Emergency fund coverage:',
            'options': {
                'None': 1,
                '1-3 months': 3,
                '3-6 months': 5,
                '6+ months': 7
            },
            'weight': 0.15
        },
        'investment_goal': {
            'question': 'Primary investment goal:',
            'options': {
                'Preserve capital': 1,
                'Generate income': 3,
                'Balanced growth': 5,
                'Maximum growth': 7
            },
            'weight': 0.10
        }
    }
    
    return questions

def calculate_risk_score(responses):
    """
    Calculate overall risk tolerance score.
    
    Parameters:
    -----------
    responses : dict
        Dictionary mapping question keys to selected option scores
    
    Returns:
    --------
    tuple : (score, profile, recommended_allocation)
    """
    questions = assess_risk_tolerance()
    
    weighted_score = 0
    for key, score in responses.items():
        weighted_score += score * questions[key]['weight']
    
    # Normalize to 0-100 scale
    risk_score = (weighted_score / 7) * 100
    
    # Determine risk profile
    if risk_score < 30:
        profile = 'Conservative'
        allocation = {'stocks': 30, 'bonds': 60, 'cash': 10}
    elif risk_score < 50:
        profile = 'Moderately Conservative'
        allocation = {'stocks': 50, 'bonds': 45, 'cash': 5}
    elif risk_score < 70:
        profile = 'Moderate'
        allocation = {'stocks': 70, 'bonds': 28, 'cash': 2}
    elif risk_score < 85:
        profile = 'Moderately Aggressive'
        allocation = {'stocks': 85, 'bonds': 14, 'cash': 1}
    else:
        profile = 'Aggressive'
        allocation = {'stocks': 95, 'bonds': 5, 'cash': 0}
    
    return risk_score, profile, allocation

# Example assessment
sample_responses = {
    'time_horizon': 5,      # 10-20 years
    'portfolio_drop': 5,    # Hold steady
    'income_stability': 5,  # Stable
    'emergency_fund': 5,    # 3-6 months
    'investment_goal': 5    # Balanced growth
}

score, profile, allocation = calculate_risk_score(sample_responses)

print("Risk Tolerance Assessment Results")
print("=" * 50)
print(f"\nRisk Score: {score:.1f}/100")
print(f"Risk Profile: {profile}")
print("\nRecommended Asset Allocation:")
for asset, pct in allocation.items():
    print(f"  {asset.capitalize()}: {pct}%")

In [ ]:
# Visualize risk profiles and allocations
profiles = [
    ('Conservative', {'stocks': 30, 'bonds': 60, 'cash': 10}),
    ('Mod. Conservative', {'stocks': 50, 'bonds': 45, 'cash': 5}),
    ('Moderate', {'stocks': 70, 'bonds': 28, 'cash': 2}),
    ('Mod. Aggressive', {'stocks': 85, 'bonds': 14, 'cash': 1}),
    ('Aggressive', {'stocks': 95, 'bonds': 5, 'cash': 0})
]

fig, axes = plt.subplots(1, 5, figsize=(18, 4))
colors = ['#2E7D32', '#1565C0', '#F57C00']

for idx, (profile_name, allocation) in enumerate(profiles):
    ax = axes[idx]
    
    # Create pie chart
    values = [allocation['stocks'], allocation['bonds'], allocation['cash']]
    labels = ['Stocks', 'Bonds', 'Cash']
    
    # Only include non-zero values
    non_zero = [(l, v, c) for l, v, c in zip(labels, values, colors) if v > 0]
    if non_zero:
        labels_nz, values_nz, colors_nz = zip(*non_zero)
        
        wedges, texts, autotexts = ax.pie(values_nz, labels=labels_nz, colors=colors_nz,
                                           autopct='%1.0f%%', startangle=90,
                                           textprops={'fontsize': 9})
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
    
    ax.set_title(profile_name, fontsize=11, fontweight='bold')

plt.suptitle('Asset Allocation by Risk Profile', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nRisk Profile Characteristics:")
print("-" * 50)
print("Conservative: Capital preservation, low volatility")
print("Mod. Conservative: Income focus, some growth")
print("Moderate: Balanced growth and income")
print("Mod. Aggressive: Growth focus, accepts volatility")
print("Aggressive: Maximum growth, high volatility tolerance")

## Part 3: Asset Class Correlations

Correlation measures how assets move together:

```
Correlation Coefficient:
========================

+1.0  = Perfect positive correlation (move together)
 0.0  = No correlation (independent)
-1.0  = Perfect negative correlation (move opposite)

Diversification Benefits:
=========================

Low/Negative Correlation → Better Diversification
High Positive Correlation → Less Diversification

Example Correlations (Historical):
-----------------------------------
US Stocks vs International Stocks:  +0.85
US Stocks vs US Bonds:              +0.10
Stocks vs Gold:                     -0.10
Bonds vs Real Estate:               +0.20
```

In [ ]:
# Simulate historical returns for different asset classes
np.random.seed(42)
n_years = 20
n_months = n_years * 12

# Generate correlated returns
dates = pd.date_range(start='2004-01-01', periods=n_months, freq='M')

# Create correlation structure
correlation_matrix = np.array([
    [1.00, 0.85, 0.10, -0.05, 0.60, 0.30],  # US Stocks
    [0.85, 1.00, 0.15, -0.10, 0.70, 0.40],  # International Stocks
    [0.10, 0.15, 1.00,  0.20, 0.05, 0.25],  # US Bonds
    [-0.05, -0.10, 0.20, 1.00, -0.15, 0.10],  # Gold
    [0.60, 0.70, 0.05, -0.15, 1.00, 0.50],  # Real Estate
    [0.30, 0.40, 0.25, 0.10, 0.50, 1.00]   # Commodities
])

# Generate returns using Cholesky decomposition
L = np.linalg.cholesky(correlation_matrix)
uncorrelated = np.random.normal(0, 1, (n_months, 6))
correlated = uncorrelated @ L.T

# Scale to realistic returns (annualized mean and std)
asset_params = {
    'US Stocks': (0.10, 0.18),
    'Intl Stocks': (0.08, 0.20),
    'US Bonds': (0.04, 0.05),
    'Gold': (0.05, 0.15),
    'Real Estate': (0.09, 0.16),
    'Commodities': (0.06, 0.20)
}

returns_data = {}
for idx, (asset, (mean_annual, std_annual)) in enumerate(asset_params.items()):
    mean_monthly = mean_annual / 12
    std_monthly = std_annual / np.sqrt(12)
    returns_data[asset] = correlated[:, idx] * std_monthly + mean_monthly

returns_df = pd.DataFrame(returns_data, index=dates)

# Calculate correlation matrix
correlation = returns_df.corr()

print("Asset Class Correlation Matrix")
print("=" * 50)
print(correlation.round(2))

In [ ]:
# Visualize correlation matrix as heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(correlation, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            vmin=-1, vmax=1, ax=ax)

ax.set_title('Asset Class Correlation Matrix\n(Monthly Returns)', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("-" * 50)
print("Green = Positive correlation (move together)")
print("Red = Negative correlation (move opposite)")
print("Yellow = Low/no correlation (independent)")
print("\nBest diversification: Combine assets with low/negative correlations")

In [ ]:
# Calculate annualized statistics
annual_returns = returns_df.mean() * 12 * 100
annual_volatility = returns_df.std() * np.sqrt(12) * 100
sharpe_ratio = (annual_returns - 2) / annual_volatility  # Assume 2% risk-free rate

stats_df = pd.DataFrame({
    'Annual Return (%)': annual_returns,
    'Annual Volatility (%)': annual_volatility,
    'Sharpe Ratio': sharpe_ratio
}).round(2)

print("\nAsset Class Statistics (Annualized)")
print("=" * 50)
print(stats_df)

# Visualize risk-return tradeoff
fig, ax = plt.subplots(figsize=(12, 7))

colors_map = ['#D32F2F', '#F57C00', '#1976D2', '#FBC02D', '#388E3C', '#7B1FA2']

for idx, asset in enumerate(returns_df.columns):
    ax.scatter(annual_volatility[asset], annual_returns[asset], 
              s=300, alpha=0.7, color=colors_map[idx], edgecolors='black', linewidth=2)
    ax.annotate(asset, (annual_volatility[asset], annual_returns[asset]),
               xytext=(10, 5), textcoords='offset points', fontsize=10, fontweight='bold')

ax.set_xlabel('Annual Volatility (Risk) %', fontsize=12, fontweight='bold')
ax.set_ylabel('Annual Return %', fontsize=12, fontweight='bold')
ax.set_title('Asset Class Risk-Return Tradeoff', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(y=2, color='gray', linestyle='--', alpha=0.5, label='Risk-free rate')
ax.legend()

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("Assets higher and to the left have better risk-adjusted returns (higher Sharpe ratio)")

## Part 4: Portfolio Allocation Strategies

### Modern Portfolio Theory (MPT)

Harry Markowitz won a Nobel Prize for showing that:

```
Modern Portfolio Theory:
========================

1. Efficient Frontier
   - Set of portfolios with maximum return for given risk
   - Or minimum risk for given return
   
2. Diversification Benefits
   - Portfolio risk < sum of individual asset risks
   - Due to imperfect correlations
   
3. Risk-Return Optimization
   - Find optimal weights to maximize Sharpe ratio
   - Balance return vs volatility

Common Portfolio Strategies:
============================

1. Equal Weight
   - 1/N allocation to each asset
   - Simple but ignores risk differences
   
2. Market Cap Weight
   - Weight by market size
   - Follows index approach
   
3. Risk Parity
   - Equal risk contribution from each asset
   - Allocate more to lower-volatility assets
   
4. Mean-Variance Optimization
   - Maximize risk-adjusted return
   - Requires return forecasts (challenging)
```

In [ ]:
def calculate_portfolio_stats(weights, returns_df):
    """
    Calculate portfolio statistics given weights.
    
    Parameters:
    -----------
    weights : array
        Portfolio weights (must sum to 1)
    returns_df : DataFrame
        Historical returns
    
    Returns:
    --------
    tuple : (annual_return, annual_volatility, sharpe_ratio)
    """
    # Portfolio returns
    portfolio_returns = (returns_df * weights).sum(axis=1)
    
    # Annualized statistics
    annual_return = portfolio_returns.mean() * 12 * 100
    annual_vol = portfolio_returns.std() * np.sqrt(12) * 100
    sharpe = (annual_return - 2) / annual_vol
    
    return annual_return, annual_vol, sharpe

def optimize_portfolio(returns_df, target='sharpe', target_return=None):
    """
    Optimize portfolio allocation.
    
    Parameters:
    -----------
    returns_df : DataFrame
        Historical returns
    target : str
        'sharpe' for max Sharpe ratio, 'variance' for min variance
    target_return : float
        Target return if optimizing for min variance at given return
    
    Returns:
    --------
    array : Optimal weights
    """
    n_assets = len(returns_df.columns)
    
    def neg_sharpe(weights):
        ret, vol, sharpe = calculate_portfolio_stats(weights, returns_df)
        return -sharpe
    
    def portfolio_variance(weights):
        _, vol, _ = calculate_portfolio_stats(weights, returns_df)
        return vol
    
    # Constraints: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Bounds: 0 <= weight <= 1
    bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Initial guess: equal weight
    init_weights = np.array([1/n_assets] * n_assets)
    
    if target == 'sharpe':
        result = minimize(neg_sharpe, init_weights, method='SLSQP',
                        bounds=bounds, constraints=constraints)
    else:
        result = minimize(portfolio_variance, init_weights, method='SLSQP',
                        bounds=bounds, constraints=constraints)
    
    return result.x

# Calculate different allocation strategies
strategies = {}

# 1. Equal weight
n_assets = len(returns_df.columns)
strategies['Equal Weight'] = np.array([1/n_assets] * n_assets)

# 2. Max Sharpe ratio
strategies['Max Sharpe'] = optimize_portfolio(returns_df, target='sharpe')

# 3. Minimum variance
strategies['Min Variance'] = optimize_portfolio(returns_df, target='variance')

# 4. Traditional 60/40 (60% stocks, 40% bonds)
traditional_60_40 = np.zeros(n_assets)
traditional_60_40[0] = 0.60  # US Stocks
traditional_60_40[2] = 0.40  # US Bonds
strategies['Traditional 60/40'] = traditional_60_40

# Calculate stats for each strategy
print("Portfolio Allocation Strategies")
print("=" * 80)
print(f"\n{'Strategy':<20} {'Return':<10} {'Volatility':<12} {'Sharpe':<8} Allocation")
print("-" * 80)

strategy_stats = {}
for name, weights in strategies.items():
    ret, vol, sharpe = calculate_portfolio_stats(weights, returns_df)
    strategy_stats[name] = (ret, vol, sharpe)
    
    # Format allocation
    allocation_str = ', '.join([f"{w*100:.0f}%" for w in weights if w > 0.01])
    
    print(f"{name:<20} {ret:>6.2f}%    {vol:>6.2f}%      {sharpe:>5.2f}  {allocation_str}")

print("\n" + "=" * 80)

In [ ]:
# Visualize allocation strategies
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

asset_names = returns_df.columns
colors = ['#D32F2F', '#F57C00', '#1976D2', '#FBC02D', '#388E3C', '#7B1FA2']

for idx, (name, weights) in enumerate(strategies.items()):
    ax = axes[idx]
    
    # Filter out zero weights
    non_zero_idx = weights > 0.01
    weights_nz = weights[non_zero_idx]
    names_nz = asset_names[non_zero_idx]
    colors_nz = [colors[i] for i, nz in enumerate(non_zero_idx) if nz]
    
    # Create pie chart
    wedges, texts, autotexts = ax.pie(weights_nz, labels=names_nz, colors=colors_nz,
                                       autopct='%1.1f%%', startangle=90,
                                       textprops={'fontsize': 9})
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    # Add statistics
    ret, vol, sharpe = strategy_stats[name]
    stats_text = f"Return: {ret:.2f}%\nVolatility: {vol:.2f}%\nSharpe: {sharpe:.2f}"
    ax.text(0, -1.4, stats_text, ha='center', fontsize=10,
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.set_title(name, fontsize=12, fontweight='bold', pad=10)

plt.suptitle('Comparison of Portfolio Allocation Strategies', 
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
# Generate efficient frontier
def generate_efficient_frontier(returns_df, n_portfolios=100):
    """
    Generate efficient frontier by optimizing for different return levels.
    """
    results = []
    
    # Generate random portfolios for comparison
    n_assets = len(returns_df.columns)
    for _ in range(1000):
        weights = np.random.random(n_assets)
        weights /= weights.sum()
        ret, vol, sharpe = calculate_portfolio_stats(weights, returns_df)
        results.append({'return': ret, 'volatility': vol, 'sharpe': sharpe, 'type': 'random'})
    
    # Add optimized portfolios
    for name, weights in strategies.items():
        ret, vol, sharpe = calculate_portfolio_stats(weights, returns_df)
        results.append({'return': ret, 'volatility': vol, 'sharpe': sharpe, 'type': name})
    
    return pd.DataFrame(results)

frontier_df = generate_efficient_frontier(returns_df)

# Plot efficient frontier
fig, ax = plt.subplots(figsize=(14, 8))

# Plot random portfolios
random = frontier_df[frontier_df['type'] == 'random']
scatter = ax.scatter(random['volatility'], random['return'], 
                    c=random['sharpe'], cmap='viridis', alpha=0.3, s=20)

# Plot optimized strategies
strategy_colors = {
    'Equal Weight': 'blue',
    'Max Sharpe': 'red',
    'Min Variance': 'green',
    'Traditional 60/40': 'orange'
}

for name in strategies.keys():
    strategy_point = frontier_df[frontier_df['type'] == name]
    ax.scatter(strategy_point['volatility'], strategy_point['return'],
              s=300, marker='*', color=strategy_colors[name], 
              edgecolors='black', linewidth=2, label=name, zorder=5)

# Add colorbar for Sharpe ratio
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Sharpe Ratio', fontsize=11, fontweight='bold')

ax.set_xlabel('Annual Volatility (Risk) %', fontsize=12, fontweight='bold')
ax.set_ylabel('Annual Return %', fontsize=12, fontweight='bold')
ax.set_title('Efficient Frontier: Risk vs Return\nOptimized Strategies vs Random Portfolios', 
             fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEfficient Frontier Interpretation:")
print("=" * 50)
print("- Random portfolios (dots) show all possible combinations")
print("- Color indicates Sharpe ratio (darker = better risk-adjusted return)")
print("- Stars show optimized strategies")
print("- Max Sharpe portfolio offers best risk-adjusted return")
print("- Min Variance portfolio offers lowest risk")

## Part 5: Practical Portfolio Construction

### Building Your Allocation

Let's combine everything we've learned to build a practical portfolio:

In [ ]:
def build_custom_portfolio(age, risk_tolerance_score, portfolio_value):
    """
    Build a custom portfolio based on investor profile.
    
    Parameters:
    -----------
    age : int
        Investor age
    risk_tolerance_score : float
        Risk tolerance score (0-100)
    portfolio_value : float
        Total portfolio value in dollars
    
    Returns:
    --------
    dict : Portfolio allocation and dollar amounts
    """
    # Start with age-based allocation (110 rule)
    base_stock_pct = min(90, max(30, 110 - age))
    
    # Adjust based on risk tolerance
    risk_adjustment = (risk_tolerance_score - 60) * 0.3  # Neutral at 60
    adjusted_stock_pct = max(20, min(95, base_stock_pct + risk_adjustment))
    
    # Calculate bond and cash allocations
    remaining = 100 - adjusted_stock_pct
    bond_pct = remaining * 0.9  # 90% of remaining
    cash_pct = remaining * 0.1  # 10% of remaining
    
    # Break down stock allocation
    stock_allocation = {
        'US Large Cap': adjusted_stock_pct * 0.50,
        'US Small/Mid Cap': adjusted_stock_pct * 0.15,
        'International': adjusted_stock_pct * 0.25,
        'Emerging Markets': adjusted_stock_pct * 0.10
    }
    
    # Break down bond allocation
    bond_allocation = {
        'US Investment Grade': bond_pct * 0.70,
        'US TIPS': bond_pct * 0.20,
        'International Bonds': bond_pct * 0.10
    }
    
    # Combine all allocations
    full_allocation = {**stock_allocation, **bond_allocation, 'Cash/Money Market': cash_pct}
    
    # Calculate dollar amounts
    dollar_allocation = {k: (v/100) * portfolio_value for k, v in full_allocation.items()}
    
    return {
        'percentages': full_allocation,
        'dollars': dollar_allocation,
        'summary': {
            'total_stocks': adjusted_stock_pct,
            'total_bonds': bond_pct,
            'total_cash': cash_pct
        }
    }

# Example: 35-year-old moderate investor with $100,000
example_age = 35
example_risk_score = 65  # Moderate-to-aggressive
example_portfolio = 100000

allocation = build_custom_portfolio(example_age, example_risk_score, example_portfolio)

print("Custom Portfolio Allocation")
print("=" * 70)
print(f"\nInvestor Profile:")
print(f"  Age: {example_age}")
print(f"  Risk Score: {example_risk_score}/100")
print(f"  Portfolio Value: ${example_portfolio:,.0f}")

print(f"\nAsset Class Summary:")
print(f"  Stocks: {allocation['summary']['total_stocks']:.1f}%")
print(f"  Bonds:  {allocation['summary']['total_bonds']:.1f}%")
print(f"  Cash:   {allocation['summary']['total_cash']:.1f}%")

print(f"\nDetailed Allocation:")
print(f"{'Asset':<25} {'Percent':>10} {'Dollar Amount':>15}")
print("-" * 70)

for asset, pct in allocation['percentages'].items():
    if pct > 0:
        dollars = allocation['dollars'][asset]
        print(f"{asset:<25} {pct:>9.1f}% ${dollars:>14,.0f}")

print("=" * 70)

In [ ]:
# Visualize the custom portfolio
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Pie chart 1: High-level allocation
summary = allocation['summary']
high_level = ['Stocks', 'Bonds', 'Cash']
high_level_values = [summary['total_stocks'], summary['total_bonds'], summary['total_cash']]
colors1 = ['#2E7D32', '#1565C0', '#F57C00']

wedges, texts, autotexts = ax1.pie(high_level_values, labels=high_level, colors=colors1,
                                    autopct='%1.1f%%', startangle=90, textprops={'fontsize': 11})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)

ax1.set_title('Asset Class Allocation', fontsize=13, fontweight='bold')

# Bar chart: Detailed allocation
detailed_assets = [k for k, v in allocation['percentages'].items() if v > 0]
detailed_pcts = [allocation['percentages'][k] for k in detailed_assets]

# Color code by asset type
bar_colors = []
for asset in detailed_assets:
    if 'Cap' in asset or 'International' in asset and 'Bond' not in asset or 'Emerging' in asset:
        bar_colors.append('#2E7D32')  # Stock green
    elif 'Bond' in asset or 'TIPS' in asset:
        bar_colors.append('#1565C0')  # Bond blue
    else:
        bar_colors.append('#F57C00')  # Cash orange

y_pos = np.arange(len(detailed_assets))
bars = ax2.barh(y_pos, detailed_pcts, color=bar_colors, alpha=0.8, edgecolor='black')

# Add value labels
for i, (bar, pct) in enumerate(zip(bars, detailed_pcts)):
    width = bar.get_width()
    ax2.text(width + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{pct:.1f}%', ha='left', va='center', fontsize=9, fontweight='bold')

ax2.set_yticks(y_pos)
ax2.set_yticklabels(detailed_assets, fontsize=10)
ax2.set_xlabel('Allocation (%)', fontsize=11, fontweight='bold')
ax2.set_title('Detailed Asset Breakdown', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.suptitle(f'Custom Portfolio for {example_age}-Year-Old Investor\nRisk Score: {example_risk_score}/100', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Quiz: Asset Allocation

Test your understanding of asset allocation concepts:

In [ ]:
# Interactive quiz
quiz_questions = [
    {
        'question': "A 40-year-old investor using the '110 minus age' rule should have what stock allocation?",
        'options': ['60%', '70%', '80%', '90%'],
        'correct': 1,
        'explanation': "110 - 40 = 70% stocks, 30% bonds. This moderate rule balances growth potential with risk management."
    },
    {
        'question': "Two assets with a correlation of +0.95 provide:",
        'options': [
            'Excellent diversification',
            'Moderate diversification',
            'Minimal diversification',
            'Negative diversification'
        ],
        'correct': 2,
        'explanation': "High positive correlation (+0.95) means assets move together, providing minimal diversification benefits."
    },
    {
        'question': "Which factor is MOST important in determining asset allocation?",
        'options': [
            'Current market conditions',
            'Recent investment performance',
            'Time horizon and risk tolerance',
            'Tax bracket'
        ],
        'correct': 2,
        'explanation': "Time horizon and risk tolerance are fundamental to asset allocation. Market timing and recent performance are less reliable guides."
    },
    {
        'question': "An investor with 30 years until retirement should generally:",
        'options': [
            'Hold mostly cash for safety',
            'Hold 50/50 stocks and bonds',
            'Hold mostly stocks for growth',
            'Hold mostly bonds for income'
        ],
        'correct': 2,
        'explanation': "With 30 years to invest, this investor can tolerate volatility and should emphasize stocks for long-term growth."
    },
    {
        'question': "The Sharpe ratio measures:",
        'options': [
            'Total return',
            'Risk-adjusted return',
            'Correlation',
            'Tax efficiency'
        ],
        'correct': 1,
        'explanation': "Sharpe ratio = (Return - Risk-free rate) / Volatility. It measures excess return per unit of risk."
    },
    {
        'question': "What is the main benefit of international stock diversification?",
        'options': [
            'Higher guaranteed returns',
            'Lower correlation with US stocks',
            'No currency risk',
            'Better tax treatment'
        ],
        'correct': 1,
        'explanation': "International stocks have imperfect correlation with US stocks (typically 0.7-0.9), providing diversification benefits."
    },
    {
        'question': "An aggressive investor age 30 might use which allocation rule?",
        'options': [
            '100 minus age',
            '110 minus age',
            '120 minus age',
            'Age minus 20'
        ],
        'correct': 2,
        'explanation': "120 - 30 = 90% stocks, suitable for aggressive investors with long time horizons. The 100 and 110 rules are more conservative."
    },
    {
        'question': "The efficient frontier represents portfolios that:",
        'options': [
            'Have the highest returns',
            'Have the lowest risk',
            'Maximize return for each level of risk',
            'Follow market trends'
        ],
        'correct': 2,
        'explanation': "The efficient frontier shows portfolios with the highest expected return for each level of risk (or lowest risk for each level of return)."
    }
]

def display_quiz():
    score = 0
    print("ASSET ALLOCATION QUIZ")
    print("=" * 70)
    
    for i, q in enumerate(quiz_questions, 1):
        print(f"\nQuestion {i}: {q['question']}")
        for j, option in enumerate(q['options']):
            print(f"  {j}. {option}")
        
        # In a real interactive notebook, you'd collect user input
        # For demonstration, we'll show the correct answer
        print(f"\n  Correct Answer: {q['options'][q['correct']]}")
        print(f"  Explanation: {q['explanation']}")
        print("-" * 70)
    
    print(f"\nQuiz complete! Study these concepts to master asset allocation.")

display_quiz()

## Summary

### Key Takeaways

1. **Age-Based Rules Provide Starting Points**
   - 100/110/120 minus age formulas
   - Adjust based on personal circumstances
   - Consider longer lifespans today

2. **Risk Tolerance Has Multiple Dimensions**
   - Time horizon (ability to take risk)
   - Financial capacity (financial ability)
   - Emotional willingness (psychological ability)
   - All three must align

3. **Correlation Drives Diversification**
   - Low correlation = better diversification
   - Combine assets that move differently
   - Portfolio risk < sum of individual risks

4. **Modern Portfolio Theory Optimizes Allocations**
   - Efficient frontier maximizes risk-adjusted returns
   - Sharpe ratio measures return per unit of risk
   - Multiple valid strategies exist

5. **Practical Portfolios Balance Theory and Reality**
   - Start with age-based allocation
   - Adjust for risk tolerance
   - Diversify across and within asset classes
   - Keep it simple and maintainable

### Action Items

1. Calculate your target allocation using an age-based rule
2. Assess your personal risk tolerance honestly
3. Review your current portfolio's asset allocation
4. Identify gaps between current and target allocation
5. Plan how to reach your target (we'll cover rebalancing tomorrow)

### Tomorrow's Preview

**Day 17: Rebalancing Strategies**
- When and how to rebalance
- Calendar vs threshold methods
- Tax-efficient rebalancing techniques
- Using new contributions to rebalance

---

**Remember:** Asset allocation is the single most important investment decision. Spend time getting it right for your situation, then maintain discipline through market cycles.

### Additional Resources

- "A Random Walk Down Wall Street" by Burton Malkiel
- "The Intelligent Asset Allocator" by William Bernstein
- Vanguard's Principles for Investing Success
- Modern Portfolio Theory papers by Harry Markowitz